In [1]:
import numpy as np
import torch 
import torch.nn as nn
import pandas as pd
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold

In [2]:
path = '/content/drive/MyDrive/python_data/社群網路與推薦系統/hw3/data/Movielens/user_movie.dat'
device= 'cuda' if torch.cuda.is_available() else 'cpu'

# User-Item-Matrix

In [3]:
def get_filtered_data(path):

  names = ['user_id', 'item_id', 'rating']
  df = pd.read_csv(path, sep='\t', names=names, usecols = [i for i in range(3)])
  
  '''filtering'''
  grouped_df = df.groupby(['user_id'])['item_id'].count()
  filtered_user_id = grouped_df[grouped_df>3].index
  filtered_df = df.set_index('user_id').loc[filtered_user_id].reset_index()
  filtered_data= np.array(filtered_df)

  return filtered_data

In [4]:
def get_mask(data):
  # data 2d array with [n, 3]
  mask = torch.zeros(size= (n_user, n_item), dtype= float, device= device)
  for info in data:
    mask[info[0]-1, info[1]-1] = 1
  return mask

In [5]:
filtered_data = get_filtered_data(path= path)

In [6]:
global n_user, n_item, y
n_user= filtered_data[:,0].max()
n_item= filtered_data[:,1].max()

In [7]:
y = torch.zeros(size= (n_user, n_item), dtype= float, device= device) 
for info in filtered_data:
  y[info[0]-1, info[1]-1] = info[2]
print(y.size())

torch.Size([943, 1682])


----

# MF

In [8]:
class MF(nn.Module):
  def __init__(self, n_user, n_item, k, beta):
    super(MF, self).__init__()
    self.n_user= n_user
    self.n_item= n_item
    self.beta = beta
    """parameters | .to(device) creates new tensor so that the params cant be fetched from model.parameters()"""
    self.b = torch.nn.Parameter(torch.zeros(size= (1, 1), dtype= float, device= device))
    self.p = torch.nn.Parameter(torch.randn(size= (self.n_user, k), dtype= float, device= device))
    self.q = torch.nn.Parameter(torch.randn(size= (self.n_item, k), dtype= float, device= device))
    self.b_u = torch.nn.Parameter(torch.zeros(size=(self.n_user, 1), dtype= float, device= device))
    self.b_i = torch.nn.Parameter(torch.zeros(size=(self.n_item, 1), dtype= float, device= device)) 
    # self.params = nn.ParameterList([self.p, self.q, self.b_u, self.b_i])
  
  def forward(self):
    predict = self.b + self.b_u.repeat(1, self.n_item) + self.b_i.t().repeat(self.n_user, 1) + torch.matmul(self.p, self.q.t())
    return predict

  def loss_function(self, predict, y_mask):

    mask_boolean = (y_mask > 0)

    sse = torch.sum((torch.masked_select(y, mask_boolean) - torch.masked_select(predict, mask_boolean))**2)
    p_norm = torch.norm(self.p, dim= 1).unsqueeze(dim= 1) #[n_user, 1]
    q_norm = torch.norm(self.q, dim= 1).unsqueeze(dim= 1) #[n_item, 1]
    sum_b_u = torch.sum((torch.masked_select(self.b_u.repeat(1, self.n_item), mask_boolean))**2)
    sum_b_i = torch.sum((torch.masked_select(self.b_i.t().repeat(self.n_user, 1), mask_boolean))**2)
    sum_b = ((self.b**2) * torch.sum(y_mask)).squeeze(dim= 1)
    sum_p_norm = torch.sum((torch.masked_select(p_norm.repeat(1, self.n_item), mask_boolean))**2)
    sum_q_norm = torch.sum((torch.masked_select(q_norm.t().repeat(self.n_user, 1), mask_boolean))**2)

    # print(sse.size())
    # print(sum_b_u.size())
    # print(sum_b_i.size())
    # print(sum_b.size())
    # print(sum_p_norm.size())
    # print(sum_q_norm.size())
    return 0.5*sse + self.beta * 0.5 * (sum_b_u + sum_b_i + sum_b + sum_p_norm + sum_q_norm)
  
  def RMSE(self, predict, test_mask): 
    # test_mask: [n_user, n_item] float
    # predict: [n_user, n_item] float

    test_mask = (test_mask > 0)

    return round(mean_squared_error(torch.masked_select(y, test_mask).detach().cpu(), torch.masked_select(predict, test_mask).detach().cpu(), squared= False), 2)

# Training 

In [14]:
lr= 5e-2
iteration = 100
k= 4
beta = 0.001

In [15]:
model = MF(n_user= n_user, n_item= n_item, k=k, beta=beta)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
criterion = nn.MSELoss()
kf= KFold(n_splits=5)

In [17]:
for epoch in range(iteration):
  RMSEs = []
  for train_indices, test_indices in kf.split(filtered_data):
    train_y_mask= get_mask(data= filtered_data[train_indices])
    test_y_mask= get_mask(data= filtered_data[test_indices])
    
    '''training process'''
    model.train()
    optimizer.zero_grad()
    output= model()
    # loss = model.loss_function(predict= output, y_mask= train_y_mask)
    # print(f'loss: {round(loss.item(), 2)}')
    loss = criterion(output, y*train_y_mask)
    loss.backward()
    optimizer.step()

    """testing process"""

    rmse = model.RMSE(predict= output, test_mask= test_y_mask)
    RMSEs.append(rmse)
  
  if ((epoch+1)%10) == 0: 
    print(f'Avg RMSE(test): {np.mean(RMSEs)}')
    print(f'loss: {round(loss.item(), 2)}')

KeyboardInterrupt: ignored